In [1]:
from langchain_openai import ChatOpenAI
import os
import openai
import getpass
import tiktoken
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='e약은요_의약품명추가.csv')
data = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, length_function=tiktoken_len)
texts = text_splitter.split_documents(data)

In [6]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model= "text-embedding-3-large")

In [7]:
#save to disk
emed_db = Chroma.from_documents(texts, embedding,persist_directory="./e약은요db_openai_3_large_의약품명추가")

In [8]:
#벡터db의 데이터가 in-memory가 아니라 persistent storage인 disk에 저장되게 선언
emed_db.persist()
emed_db = None

In [9]:
# load from disk
emed_db = Chroma(persist_directory="./e약은요db_openai_3_large_의약품명추가",embedding_function=embedding)

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

openai = ChatOpenAI(model = "gpt-3.5-turbo",temperature=0)

retriever = emed_db.as_retriever(search_type="similarity", search_kwargs={'k':3})#search_type = "mmr",

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    openai, retriever, contextualize_q_prompt
)

#답변 생성
qa_system_prompt = """You are an assistant for question-answering tasks. \
ONLY USE the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(openai, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

# chat history 관리
store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key= "input",
    history_messages_key= "chat_history",
    output_messages_key= "answer",
)


c:\Users\Admin\anaconda3\envs\sba\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
id = "0"
query1 = "게보린정 효능 알려줘"
conversational_rag_chain.invoke(
    {"input": query1},
    config={
        "configurable": {"session_id":id}
    },
)["answer"]

'죄송합니다, 게보린정에 대한 정보는 제공할 수 없습니다.'

# Test 수행

In [12]:
import pandas as pd

In [13]:
test_data = pd.read_excel("data preprocessing\e약은요 테스트용 질문 데이터셋.xlsx")
test_data.head()

,question,answer
0,닥터베아제정의 효능은 무엇입니까?,"이 약은 소화불량, 식욕감퇴(식욕부진), 과식, 체함, 소화촉진, 소화불량으로 인한..."
1,닥터베아제정은 어떻게 사용합니까?,성인 1회 1정을 1일 3회 식후에 복용합니다.
2,닥터베아제정을 사용하기 전에 반드시 알아야 할 내용은 무엇입니까?,NaN
3,닥터베아제정의 사용상 주의사항은 무엇입니까?,만 7세 이하의 소아는 이 약을 복용하지 마십시오.이 약을 복용하기 전에 알레르기 ...
4,닥터베아제정을 사용하는 동안 주의해야 할 약 또는 음식은 무엇입니까?,NaN


In [18]:
len(test_data)

686

### 참고사항
추후에 특정 의약품에 대한 질문을 이어서했을때의 정확도를 살피고자 할때는 session_id로 제공되는 부분에 의약품명을 넣으면 된다.

의약품명을 넣는 방법은 애초에 데이터셋을 만들때 의약품명 열을 하나 추가하면 된다

In [14]:
import time
def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"{func.__name__} 실행 시간: {elapsed_time} 초")
        return result
    return wrapper

In [15]:
@measure_time
def invoke(id, input):
    answer = conversational_rag_chain.invoke(
        {"input": input},
        config={
            "configurable": {"session_id":id}
        },
    )["answer"]

    return answer

In [16]:
answer = [] #llm의 응답을 저장할 리스트
for i, row in test_data.iterrows():
    answer.append(invoke(row.question, row.question))
    print(i+1,'번째 답변 : ',answer[i])

invoke 실행 시간: 2.3237085342407227 초
1 번째 답변 :  이 약은 소화불량, 식욕감퇴(식욕부진), 과식, 체함, 소화촉진, 소화불량으로 인한 위부팽만감에 사용합니다.
invoke 실행 시간: 1.6030917167663574 초
2 번째 답변 :  성인 1회 1정을 1일 3회 식후에 복용합니다.
invoke 실행 시간: 4.086785554885864 초
3 번째 답변 :  만 7세 이하의 소아는 이 약을 복용하지 마십시오. 이 약을 복용하기 전에 알레르기 체질인 사람, 임부 또는 임신하고 있을 가능성이 있는 여성, 이 약 또는 황색4호에 과민증 환자는 의사 또는 약사와 상담하십시오. 정해진 용법과 용량을 잘 지키십시오.
invoke 실행 시간: 3.4123618602752686 초
4 번째 답변 :  만 7세 이하의 소아는 이 약을 복용하지 마십시오. 이 약을 복용하기 전에 알레르기 체질인 사람, 임부 또는 임신하고 있을 가능성이 있는 여성, 이 약 또는 황색4호에 과민증 환자는 의사 또는 약사와 상담하십시오. 정해진 용법과 용량을 잘 지키십시오.
invoke 실행 시간: 3.3989877700805664 초
5 번째 답변 :  닥터베아제정을 사용하는 동안 다른 약물을 복용하고 있는 경우 의사 또는 약사와 상의해야 합니다. 특히 알레르기 체질인 환자, 임부 또는 임신 가능성이 있는 여성은 복용 전에 의사 또는 약사와 상담해야 합니다.
invoke 실행 시간: 13.100683689117432 초
6 번째 답변 :  쇽(호흡곤란, 혈압저하 등), 헤마토크리트감소, 헤모글로빈감소, 빈혈, 재생불량성빈혈, 용혈성빈혈, 무과립구증, 과립구감소, 백혈구감소, 혈소판감소, 혈소판기능저하, 호산구증가 등의 혈액장애, 소화성궤양, 위장관궤양, 위장출혈, 천공(뚫림), 궤양성대장염, 혈변, 위염, 췌장염, 혈액구토, 크론병, 식욕부진, 구역, 구토, 복통, 소화불량, 설사, 위부불쾌감, 상복부통증, 구갈, 구내염, 복부팽만감, 구강궤양, 변비, 흑변

In [17]:
question = test_data['question']
test_output = pd.DataFrame({"question": question,
                            "answer" : answer})
test_output.to_excel("e약은요 4차 테스트 결과_openai_3_large_문서3개.xlsx",index = False)

### db 초기화
```python
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()
```